vote

In [1]:
import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')

In [2]:
# lib
import os
import gc
import random
import math
import time

import albumentations as A
import copy
import cv2
import numpy as np
import pandas as pd

from albumentations.pytorch.transforms import ToTensorV2
from glob import glob
from scipy import stats

import torch
import torch.nn as nn
import torch.nn.functional as F
import timm
from torch.utils.data import DataLoader, Dataset
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')

#import warnings
#warnings.filterwarnings("ignore")

from fastprogress import master_bar, progress_bar


def seed_everything(seed=777):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    #torch.backends.cudnn.deterministic = True

SEED = 777
seed_everything(SEED)

torch.__version__

'1.7.0'

In [3]:
data_path = '../input/cassava-leaf-disease-classification/'

class CFG:
    device = device
    debug = False
    
    # data set 
    size = 512
    cat_nums = None
    bs = 16
    pre_load = True
    worker = 0    

    # model
    epochs = 15
    model_name = 'tf_efficientnet_b2_ns'
    hidden_size = 256
    dropout = 0.5
    
    # gradient
    iters_to_accumulate = 1
    max_grad_norm = 0.5
    criterion = nn.CrossEntropyLoss()
    metric = None
    
    # optimizer
    lr = 3e-4
    wd = 1e-6
    patience = 4
    optimizer = torch.optim.Adam
    optim_param = {'lr': lr, 'weight_decay': wd}
    scheduler = torch.optim.lr_scheduler.LambdaLR
    scheduler_param = {'lr_lambda': lambda epoch: 1}
    
    # TTA
    tta_nums = 9
    
if CFG.debug: data_path = '../input/debug/'

In [4]:
aug = A.Compose([
    A.Transpose(p=0.5),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.ShiftScaleRotate(p=0.5),
     A.HueSaturationValue(hue_shift_limit=5, 
                             sat_shift_limit=5, 
                             val_shift_limit=5),
    A.RandomGamma(),
    
])



def get_transforms(aug=None, size=CFG.size):
    resize = A.OneOf([
        #A.Resize(size, size, p=0.1),
        #A.RandomCrop(size, size, p=0.9),
        A.RandomResizedCrop(size, size),
    ], p=1.0) if aug else A.Resize(size, size)

    return A.Compose(
        [
            resize, 
            aug,
            A.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2()
        ],
        p=1.0,
    )


class TestDataset(Dataset):
    def __init__(self, test_df, image_paths, transform=None):
        self.test_names = test_df.image_id.values
        self.image_paths = image_paths
        self.transform = get_transforms(transform)

    def __len__(self):
        return len(self.test_names)

    def __getitem__(self, idx):
        img_name = self.test_names[idx]
        x = cv2.imread(self.image_paths + img_name)[..., ::-1].copy()
        #x = self.transform(image=x)['image']  # .to(device)
        return x, img_name

In [5]:
class PredictModel(nn.Module):
    def __init__(self,
                 model_name='tf_efficientnet_b0_ns',
                 model_path='./',
                 n_class=5):
        super().__init__()
        # create single
        self.model = timm.create_model(model_name,
                                       in_chans=3)
        nums_in_features = self.model.classifier.in_features
        self.model.classifier = nn.Linear(nums_in_features, n_class)
        self.freeze()
        self.model.eval().to(device)

        # load checkpoints
        checkpoints = [torch.load(path, map_location=torch.device('cpu'))
                       for path in glob(model_path + 'fold*.pth')]
        # create models
        self.models = [copy.deepcopy(self.model)
                       for _ in range(len(checkpoints))]
        # load weights
        for idx, checkpoint in enumerate(checkpoints):
            self.models[idx].load_state_dict(checkpoint['model_state_dict'])

    def forward(self, x):
        with torch.no_grad():
            x = torch.mean(torch.cat([model(x)[None]
                                      for model in self.models], dim=0), dim=0)
        return x

    def freeze(self, n=None):
        for param in list(self.model.parameters())[:n]:
            param.requires_grad = False

    def unfreeze(self):
        for param in self.model.parameters():
            param.requires_grad = True

In [6]:
model = PredictModel(model_name=CFG.model_name,
                     model_path='../input/cld-ef-b2-089251/')

In [7]:
test_path = '../input/cassava-leaf-disease-classification/'
image_path = test_path + 'test_images/'
test_df = pd.read_csv(test_path + 'sample_submission.csv')

test_ds = TestDataset(test_df=test_df, 
                      image_paths=image_path, 
                      transform=None)
test_dl = DataLoader(test_ds, batch_size=1)

for xs, names in test_dl:
    xs = torch.stack([get_transforms(aug)(image=xs[0].numpy())['image'] 
                      for _ in range(CFG.tta_nums)])
    xs = xs.to(device)
    preds = model(xs).detach().cpu().numpy()
    preds = stats.mode(preds.argmax(1))[0].item()
    test_df.loc[test_df.image_id.isin(names), 'label'] = preds
    
test_df

,image_id,label
0,2216849948.jpg,4


In [8]:
test_df.to_csv('submission.csv', index=False)